# ⭐️ Transformer Architecture
* ⭐️ encoder: 언어 이해
* ⭐️ decoder: 언어 생성
* ⭐️ 병렬 연산 O: 모든 입력을 동시에 처리
* 대부분 LLM(Large Language Model, 대규모 언어 모델)에서 사용: 딥러닝 기반
* 구글 논문: Attention is All you need.
* self-attention: 입력된 문장 내의 각 단어가 서로 어떤 관련이 있는지 계산.
---
### cf. RRN(Recurrent Neural Network, 순환신경망)
* 입력을 순차적⭐️으로 처리. 즉 병렬 처리 X
---

## 1️⃣ 토큰화(tokenization): 텍스트를 적절한 단위로 잘라 숫자 ID를 부여.
* 사전(vocabulary): 토큰과 매칭된 숫자 ID 기록.
* ex) 한글: (자음과 모음 < 음절 < 단어) => 작은 단위 < 큰단위
  * 큰 단위 기준으로 자른다면
    * (-) OOV(Out Of Vocabulary, 사전에 없는 단어): 사전에 없으면 처리하지 못하는 문제.
    * (-) 텍스트에 등장하는 단어 수만큼 토큰 ID가 필요하기 때문에 사전의 크기가 커짐.
    * (+) 텍스트의 의미가 잘 유지됨.

* ⭐️ subword(多): 등장 빈도에 따라 토큰화 단위 결정.
  * 자주 나오는 단어는 그대로 의미 유지, 드물게 나오는 단어는 작게 나눠 사전 크기가 커지지 않도록 함.
  * 한글의 경우 보통 음절(ex. 특수문자, 이모티콘)과 단어(ex. 유명인 이름)로 토큰화.

In [1]:
# 띄어쓰기 단위로 분리 (실습 편의상, 실무에서는 subword 주로 사용.)
input_text = '나는 최근 미국 여행을 다녀왔다'
input_text_list = input_text.split()
print(input_text_list)

['나는', '최근', '미국', '여행을', '다녀왔다']


In [2]:
# 토큰 딕셔너리
str2idx = {word: idx for idx, word in enumerate(input_text_list)}
print(str2idx)

{'나는': 0, '최근': 1, '미국': 2, '여행을': 3, '다녀왔다': 4}


In [3]:
idx2str = {idx: word for idx, word in enumerate(input_text_list)}
print(idx2str)

{0: '나는', 1: '최근', 2: '미국', 3: '여행을', 4: '다녀왔다'}


In [4]:
input_ids = [str2idx[word] for word in input_text_list]
print(input_ids)

[0, 1, 2, 3, 4]


## 2️⃣ 토큰 embedding: 데이터 -> vector(숫자 집합) 변환 (의미가 담김)


In [5]:
import torch
import torch.nn as nn

# 토큰 ID 1개 => 16차원의 벡터로 변환.
embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim) # 임베딩 층: 임의의 숫자 집합으로 변환(의미 X), 실무에선 이 층도 학습시켜서 의미를 담음.
print(embed_layer)

Embedding(5, 16)


In [6]:
token_embeddings = embed_layer(torch.tensor(input_ids))
print(token_embeddings)

tensor([[ 0.8746,  0.8480,  0.1426,  0.9157,  0.5889,  0.4193, -0.2948, -0.0175,
          1.8022, -0.2903, -1.3834,  0.1363,  0.7212,  0.2491, -1.6416,  1.5300],
        [ 0.5908, -2.9484,  2.0463,  1.2299, -0.5393,  0.4363,  0.3705, -0.4934,
          0.3782, -0.2599,  0.5239, -0.7498,  0.4881,  0.8958,  0.2389,  1.6908],
        [-1.5745, -0.6454, -0.5513,  2.2043, -0.6901,  1.1149,  0.8552,  0.1968,
         -1.2397,  0.3358, -0.3429,  0.1870,  0.3817, -0.2721, -0.0683,  0.2225],
        [ 0.0929,  1.2460,  1.2944, -0.8192,  1.1034,  0.4892,  0.7170,  0.3485,
          1.6312,  1.6057,  0.6529,  0.2096,  0.9904, -0.8546, -0.1395, -0.2887],
        [ 0.2570,  0.1427, -0.5313,  0.3440, -0.5842, -1.4011, -0.0353,  0.3135,
          0.2477,  0.8600, -0.2627,  1.2153,  0.0806, -2.5216,  1.1992, -0.2977]],
       grad_fn=<EmbeddingBackward0>)


In [7]:
token_embeddings = token_embeddings.unsqueeze(0)
print(token_embeddings)

tensor([[[ 0.8746,  0.8480,  0.1426,  0.9157,  0.5889,  0.4193, -0.2948,
          -0.0175,  1.8022, -0.2903, -1.3834,  0.1363,  0.7212,  0.2491,
          -1.6416,  1.5300],
         [ 0.5908, -2.9484,  2.0463,  1.2299, -0.5393,  0.4363,  0.3705,
          -0.4934,  0.3782, -0.2599,  0.5239, -0.7498,  0.4881,  0.8958,
           0.2389,  1.6908],
         [-1.5745, -0.6454, -0.5513,  2.2043, -0.6901,  1.1149,  0.8552,
           0.1968, -1.2397,  0.3358, -0.3429,  0.1870,  0.3817, -0.2721,
          -0.0683,  0.2225],
         [ 0.0929,  1.2460,  1.2944, -0.8192,  1.1034,  0.4892,  0.7170,
           0.3485,  1.6312,  1.6057,  0.6529,  0.2096,  0.9904, -0.8546,
          -0.1395, -0.2887],
         [ 0.2570,  0.1427, -0.5313,  0.3440, -0.5842, -1.4011, -0.0353,
           0.3135,  0.2477,  0.8600, -0.2627,  1.2153,  0.0806, -2.5216,
           1.1992, -0.2977]]], grad_fn=<UnsqueezeBackward0>)


In [8]:
token_embeddings.shape # 문장 1개, 토큰 5개(각 임베딩 16차원)

torch.Size([1, 5, 16])

## 3️⃣ 위치 인코딩(position encoding)
* 텍스트에서 순서는 매우 중요한 정보

In [9]:
# 절대적(absolute) 위치 인코딩: 토큰의 위치에 따라 고정된 임베딩을 더함. (<-> 상대적(relative) 위치 인코딩)
max_position = 12
position_embed_layer = nn.Embedding(max_position, embedding_dim) # 위치 인코딩 층
print(position_embed_layer)

Embedding(12, 16)


In [10]:
position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
print(position_ids)

tensor([[0, 1, 2, 3, 4]])


In [11]:
position_encodings = position_embed_layer(position_ids)
print(position_encodings)

tensor([[[ 4.4439e-01,  9.2950e-01,  1.0380e-01, -1.4422e-01, -1.7577e-01,
           6.5715e-01, -1.6286e+00, -2.0863e+00,  4.1216e-01, -1.9550e+00,
           3.1952e-01, -3.0065e-01,  7.5882e-01,  7.2005e-01, -1.0612e+00,
          -4.9976e-01],
         [-3.8469e-01,  3.0756e+00,  1.1635e+00,  9.0442e-01, -6.0577e-02,
          -1.4860e+00, -1.5409e+00,  5.5352e-01,  1.8478e+00,  1.4109e-01,
          -5.1490e-01,  4.2376e-01,  9.3735e-01, -2.9982e-01, -2.6732e-01,
           1.1366e+00],
         [-8.4598e-01,  1.1128e-01,  9.1669e-01, -8.8846e-01, -2.5481e-02,
           8.6478e-02,  6.9501e-01,  1.4295e+00, -1.3840e-01, -8.9193e-02,
           1.5702e+00, -1.9545e+00,  1.5986e+00, -1.0443e+00,  7.8566e-01,
          -7.3278e-01],
         [-5.1103e-01, -6.3794e-01, -7.1652e-01, -7.8443e-02,  7.2493e-03,
          -9.5150e-01,  9.3579e-01,  1.0504e+00,  2.0793e-01, -8.1841e-01,
          -1.1836e+00, -2.5159e-01,  2.0182e-03,  1.0341e+00, -1.2726e-01,
          -7.3589e-01],
    

In [12]:
input_embeddings = token_embeddings + position_encodings
print(input_embeddings) # 모델에 입력할 최종 임베딩

tensor([[[ 1.3190,  1.7775,  0.2464,  0.7714,  0.4131,  1.0765, -1.9234,
          -2.1037,  2.2144, -2.2453, -1.0638, -0.1643,  1.4800,  0.9692,
          -2.7028,  1.0302],
         [ 0.2061,  0.1272,  3.2098,  2.1344, -0.5999, -1.0497, -1.1704,
           0.0601,  2.2260, -0.1188,  0.0090, -0.3260,  1.4254,  0.5960,
          -0.0284,  2.8274],
         [-2.4205, -0.5341,  0.3654,  1.3158, -0.7156,  1.2014,  1.5502,
           1.6263, -1.3781,  0.2466,  1.2272, -1.7675,  1.9803, -1.3164,
           0.7174, -0.5103],
         [-0.4182,  0.6081,  0.5779, -0.8976,  1.1107, -0.4623,  1.6527,
           1.3989,  1.8391,  0.7873, -0.5306, -0.0420,  0.9924,  0.1794,
          -0.2668, -1.0246],
         [-0.5409,  0.1091,  0.3272,  1.2957, -1.9605, -1.5455, -1.8638,
          -1.0212, -0.0452,  1.7052,  0.0206,  2.8766, -0.7818, -0.5861,
           0.6090, -1.8746]]], grad_fn=<AddBackward0>)


In [13]:
input_embeddings.shape

torch.Size([1, 5, 16])

## 4️⃣ Attention: 사람이 단어 사이의 관계를 고민하는 과정을 딥러닝 모델이 수행할 수 있도록 모방.
* 사람: 맥락을 반영해 단어를 재해석 => ⭐️ 가중치를 사용해 토큰 간 관계를 계산해서 관련이 깊은 단어와 그렇지 않은 단어를 구분
1. query: 검색어
2. key: 문서가 가진 특징 (ex. 제목, 저자 이름, 문장 속의 각 단어)
3. value: 원하는 값

In [14]:
head_dim = 16

weight_q = nn.Linear(embedding_dim, head_dim) # 선형 층
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

querys = weight_q(input_embeddings)
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)

print(querys) # (1, 5, 16)
print(keys)
print(values)

tensor([[[-1.5116, -0.1596, -0.3667, -0.4219, -0.2443,  1.0163,  0.3633,
           0.3200,  0.7242,  0.2019, -0.6352,  0.0367, -1.3659,  0.1154,
          -0.5916, -2.3988],
         [-0.1509,  0.5546, -0.0817, -0.3647, -0.9551,  0.3279, -1.2649,
          -0.3024,  1.2079,  1.2180,  0.8141,  1.2654, -1.4511, -0.3375,
           0.9175, -2.0420],
         [ 1.1083,  0.0972,  1.0559, -0.7513, -0.1750, -1.3838, -0.5985,
          -0.1636,  1.5050,  0.3915,  1.3903,  0.6541, -0.1344, -2.0281,
          -0.5360,  1.2666],
         [ 0.1711,  0.0224,  0.2384, -1.3574, -0.5296,  0.1731, -0.0779,
          -1.2543,  0.6783, -0.1035,  0.1933,  0.0465, -0.6313, -0.3235,
          -0.0493, -0.0162],
         [ 0.1983,  0.2310,  0.1468, -0.5570, -0.6122, -0.6054,  0.5398,
           0.7238, -1.7217, -0.4450, -1.1102, -0.1528,  0.4547, -0.0673,
          -0.5439,  0.0107]]], grad_fn=<ViewBackward0>)
tensor([[[ 0.1193,  0.0346,  1.4423, -0.1546, -0.5055,  1.1183, -0.2148,
          -0.9557,  0.742

In [24]:
# 스케일 점곱 어텐션: 1개의 어텐션 연산 수행.
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1) # 16
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    if is_causal: # 디코더 - 마스크 어텐션: 작성해야 하는 텍스트를 미리 확인하게 되는 문제 해결. 즉, 미래 시점의 토큰을 제거하여 생성된 토큰까지만 확인할 수 있도록 마스크를 추가.
        query_length = querys.size(-2)
        key_length = keys.size(-2)
        temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0) # 모두 1인 행렬에 대각선 아래 부분만 1로 유지되고, 나머지는 음의 무한대로 변경해 마스크를 생성.
        scores = scores.masked_fill(temp_mask == False, float('-inf')) # 행렬의 대각선 아래 부분만 어텐션 스코어가 남고, 위쪽은 음의 무한대가 된다.
    weights = F.softmax(scores, dim=-1) # 총 합이 1 / 음의 무한대인 대각선 윗부분은 가중치가 0이 됨.
    return weights @ values

after_attention_embeddings = compute_attention(querys, keys, values) # 입력, 출력 형태 동일
print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])


In [16]:
class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        self.weight_v = nn.Linear(token_embed_dim, head_dim)

    def forward(self, querys, keys, values):
        outputs = compute_attention(
            self.weight_q(querys),
            self.weight_k(keys),
            self.weight_v(values),
            is_causal=self.is_causal
        )
        return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)
print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])


In [17]:
# 멀티 헤드 어텐션: 여러 어텐션 연산을 헤드의 수(n_head)만큼 동시에 수행, 단어 간 관계를 파악.
class MultiheadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
        super().__init__()
        self.n_head = n_head
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)

    def forward(self, querys, keys, values):
        B, T, C = querys.size()

        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        attention = compute_attention(querys, keys, values, is_causal=self.is_causal) # n_head번의 스케일 점곱 어텐션
        output = attention.transpose(1, 2).contiguous().view(B, T, C) # 어텐션 결과를 연결
        output = self.concat_linear(output) # 마지막 선형 층
        return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])


## 5️⃣ 정규화(normalization): 딥러닝 모델의 모든 입력 변수가 비슷한 범위와 분포를 갖도록 조정하면 각 입력 변수의 중요성을 공정하게 반영하여 더 정확한 예측을 할 수 있음.
* 1. 층(layer) 정규화
    - 자연어 처리
    - Transformer Architecture
    - 층과 층 사이에 정규화
    - 각 토큰 임베딩의 평균과 표준편차를 구함.
    - 사전 정규화(pre-norm)多: 먼저 층 정규화를 적용하고 어텐션과 피드 포워드 층을 통과했을 때 학습이 더 안정적. (cf. 사후 정규화(post-norm))

* 2. batch 정규화: 이미지 처리

In [21]:
# 층 정규화, 사전 정규화
norm = nn.LayerNorm(embedding_dim) # 층 정규화 레이어
norm_x = norm(input_embeddings) # 정규화된 임베딩
norm_x.shape

torch.Size([1, 5, 16])

In [22]:
norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data # 평균, 표준편차

(tensor([[ 2.6077e-08, -6.7055e-08, -2.2352e-08, -3.7253e-09, -1.4901e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

In [23]:
# feed forward layer, 완전 연결 층(fully connected layer)
# > 입력 텍스트 전체를 이해.
# > 데이터의 특징을 학습.
class PreLayerNormFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층
        self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.activation = nn.GELU()
        self.norm = nn.LayerNorm(d_model) # 층 정규화

    def forward(self, src):
        x = self.norm(src)
        x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
        x = self.dropout2(x)
        return x

## 6️⃣ 인코더

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, n_head, dim_feedforward, dropout):
        super().__init__()
        self.attn = MultiheadAttention(d_model, d_model, n_head)
        self.norm1 = nn.LayerNorm(d_model) # 층 정규화
        self.dropout1 = nn.Dropout(dropout)
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    def forward(self, src):
        norm_x = self.norm1(src)
        attn_output = self.attn(norm_x, norm_x, norm_x)
        x = src + self.dropout1(attn_output) # 잔차 연결(residual connection): 안정적인 학습이 가능하도록 도와줌.
        return self.feed_forward(x)


import copy
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)]) # 깊은 복사

class TransformerEncoder(nn.Module): # 인코더 층을 N번 반복 수행.
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, src):
        output = src
        for mod in self.layers:
            output = mod(output)
        return output


## 7️⃣ 디코더
* 사람이 글을 쓸 때 앞 단어부터 순차적으로 작성하는 것처럼 트랜스포머 모델도 앞에서 생성한 토큰을 기반으로 다음 토큰을 생성.
> 순차적으로 생성: 인과적(causal) == 자기 회귀적(auto-regressive)

In [25]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = MultiheadAttention(d_model, d_model, nhead)
        self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, tgt, encoder_output, is_causal=True):
        x = self.norm1(tgt)
        x = x + self.dropout1(self.self_attn(x, x, x, is_causal=is_causal))

        # ⭐️cross attention: 인코더의 결과를 디코더가 활용
        x = self.norm2(x)
        x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))

        return self.feed_forward(x)


class TransformerDecoder(nn.Module):
    def __init__(self, decoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(decoder_layer, num_layers) # 디코더 층을 N번 반복.
        self.num_layers = num_layers

    def forward(self, tgt, src):
        output = tgt
        for mod in self.layers:
            output = mod(output, src)
        return output